# Generating the data for analysis: baseload, detrend, ratio and log returns
    ## Cross Correlation gneration


- The baseload is the minimum level of demand: mean of the data
- Detrending is removing the effects of trend from a data set to show only the differences in values from the trend that si shown in the change in the mean over time. (by Differencing or by Model Fitting or by diff the mean)
- Ratio of the mean: number of times one value contains or is contained within the other
- Log return is used for statistical evaluation such MSPE and out-of-sample R-square.

In [16]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime
import os

In [46]:
# Generation of time series Test_df

start_time = '2020-06-01 00:00:00.00'
end_time = '2022-06-01 00:00:00.00'
start_times = pd.date_range(start= pd.Timestamp(start_time).round('60T'), end = pd.Timestamp(end_time).round('60T'), freq="60T")
end_times = start_times + pd.Timedelta('60T')
start_times_list = start_times.strftime('%Y-%m-%d %H:%M:%S').tolist()

multi_timeseries_df=pd.DataFrame()
multi_timeseries_df['datetime']=start_times_list

multi_timeseries_df['Year']=pd.DatetimeIndex(multi_timeseries_df['datetime']).year
multi_timeseries_df['Month']=pd.DatetimeIndex(multi_timeseries_df['datetime']).month
multi_timeseries_df['Day']=pd.DatetimeIndex(multi_timeseries_df['datetime']).day
multi_timeseries_df['Hour']=pd.DatetimeIndex(multi_timeseries_df['datetime']).hour

multi_timeseries_df.set_index("datetime", inplace=True)
multi_timeseries_df.index= pd.to_datetime(multi_timeseries_df.index)

for col in ['NL', 'BE', 'AT', 'DE','FR', 'CZ', 'UK']:
    multi_timeseries_df[col] = np.round((multi_timeseries_df.Day/5) *np.random.normal(-20,100, multi_timeseries_df.shape[0])+(multi_timeseries_df.Month *np.random.standard_cauchy(multi_timeseries_df.shape[0])[0]),2)
multi_timeseries_df['BE']=multi_timeseries_df['NL']/2+multi_timeseries_df['DE']/3 #adding some column corrolations
multi_timeseries_df['CZ']=multi_timeseries_df['BE']/2+multi_timeseries_df['DE']/3
multi_timeseries_df['AT']=multi_timeseries_df['AT']/2+multi_timeseries_df['DE']/2
multi_timeseries_df['FR']=multi_timeseries_df['FR']/2+multi_timeseries_df['DE']/2
multi_timeseries_df['UK']=multi_timeseries_df['UK']/2+multi_timeseries_df['FR']/2
multi_timeseries_df.head(3)

,Year,Month,Day,Hour,NL,BE,AT,DE,FR,CZ,UK
datetime,,,,,,,,,,,
2020-06-01 00:00:00,2020,6,1,0,-14.28,-12.420000,-14.825,-15.84,-22.975,-11.4900,-44.7975
2020-06-01 01:00:00,2020,6,1,1,-7.09,-5.341667,-13.785,-5.39,3.465,-4.4675,-3.2775
2020-06-01 02:00:00,2020,6,1,2,13.76,8.730000,5.720,5.55,18.490,6.2150,-7.4900


In [ ]:
multi_timeseries_df

In [47]:
multi_timeseries_df.index= pd.to_datetime(multi_timeseries_df.index)

In [48]:
def baseload_and_detrend(sliced_df: pd.DataFrame) -> pd.DataFrame:
    var_name = sliced_df.columns[0]
    detrended_name = var_name + "_detrended"
    baseload_name = var_name + "_baseload"
    ratio_name = var_name + "_ratio"

    baseload = sliced_df.groupby(by=[sliced_df.index.year, sliced_df.index.month]).mean()
    baseload.columns = [baseload_name]
    baseload.index.names = ["year", "month"]

    sliced_df["year"] = sliced_df.index.year
    sliced_df["month"] = sliced_df.index.month
    index_to_keep = sliced_df.index

    sliced_df = sliced_df.merge(baseload, on=["year", "month"])
    sliced_df[detrended_name] = sliced_df[var_name] - sliced_df[baseload_name]

    base_and_detrended_df = sliced_df.loc[:, [baseload_name, detrended_name]]
    base_and_detrended_df[ratio_name] = sliced_df[detrended_name] / sliced_df[baseload_name]
    base_and_detrended_df.set_index(index_to_keep, inplace=True)
    return base_and_detrended_df

In [49]:
for column in multi_timeseries_df.columns[4:11]:
    base_and_detrended_df = baseload_and_detrend(multi_timeseries_df.loc[:, [column]])
    multi_timeseries_df = pd.concat([multi_timeseries_df, base_and_detrended_df], axis=1)

In [50]:
multi_timeseries_df.head(3)

,Year,Month,Day,Hour,NL,BE,AT,DE,FR,CZ,...,DE_ratio,FR_baseload,FR_detrended,FR_ratio,CZ_baseload,CZ_detrended,CZ_ratio,UK_baseload,UK_detrended,UK_ratio
datetime,,,,,,,,,,,,,,,,,,,,,
2020-06-01 00:00:00,2020,6,1,0,-14.28,-12.420000,-14.825,-15.84,-22.975,-11.4900,...,-0.650833,-60.564854,37.589854,-0.620655,-40.634566,29.144566,-0.717236,-57.662747,12.865247,-0.223112
2020-06-01 01:00:00,2020,6,1,1,-7.09,-5.341667,-13.785,-5.39,3.465,-4.4675,...,-0.881186,-60.564854,64.029854,-1.057211,-40.634566,36.167066,-0.890057,-57.662747,54.385247,-0.943161
2020-06-01 02:00:00,2020,6,1,2,13.76,8.730000,5.720,5.55,18.490,6.2150,...,-1.122341,-60.564854,79.054854,-1.305293,-40.634566,46.849566,-1.152949,-57.662747,50.172747,-0.870107


In [51]:
detrended_column_names = []
baseload_column_names = []
raw_column_names = []
ratio_column_names = []

for column in multi_timeseries_df.columns[4:]:
    if "detrended" in column:
        detrended_column_names = detrended_column_names + [column]
    elif "baseload" in column:
        baseload_column_names = baseload_column_names + [column]
    elif "ratio" in column:
        ratio_column_names = ratio_column_names + [column]
    else:
        raw_column_names = raw_column_names + [column]

In [52]:
detrended_multi_timeseries_df = multi_timeseries_df.loc[:, detrended_column_names]
baseload_multi_timeseries_df = multi_timeseries_df.loc[:, baseload_column_names]
raw_multi_timeseries_df = multi_timeseries_df.loc[:, raw_column_names]
ratio_multi_timeseries_df = multi_timeseries_df.loc[:, ratio_column_names]
return_multi_timeseries_df = np.log(raw_multi_timeseries_df) - np.log(raw_multi_timeseries_df.shift(1))


# problem with hourly returns because we have negative prices.
return_multi_timeseries_df.columns = [
    "return_NL",
    "return_BE",
    "return_AT",
    "return_DE",
    "return_FR",
    "return_CZ",
    "return_UK",
]

years_in_data = detrended_multi_timeseries_df.index.year.unique()

yearly_detrended_correlations_dict = {}
yearly_baseload_correlations_dict = {}
yearly_raw_correlations_dict = {}
yearly_ratio_correlations_dict = {}
yearly_return_correlations_dict = {}

In [53]:
return_multi_timeseries_df

,return_NL,return_BE,return_AT,return_DE,return_FR,return_CZ,return_UK
datetime,,,,,,,
2020-06-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-01 02:00:00,NaN,NaN,NaN,NaN,1.674517,NaN,NaN
2020-06-01 03:00:00,NaN,NaN,0.743435,NaN,-0.566808,NaN,NaN
2020-06-01 04:00:00,NaN,NaN,-0.339719,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2022-05-31 20:00:00,NaN,NaN,2.212296,-0.584005,NaN,-1.96312,-0.034352
2022-05-31 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-31 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# YEAR DATA CORRELATIONS
for year in years_in_data:
    year_detrended = detrended_multi_timeseries_df[detrended_multi_timeseries_df.index.year == year]
    yearly_corr = year_detrended.corr()
    yearly_corr.index.names = [f"detrended_{year}"]
    yearly_corr.reset_index(inplace=True)
    yearly_detrended_correlations_dict[year] = yearly_corr

    year_baseload = baseload_multi_timeseries_df[baseload_multi_timeseries_df.index.year == year]
    yearly_corr = year_baseload.corr()
    yearly_corr.index.names = [f"baseload_{year}"]
    yearly_corr.reset_index(inplace=True)
    yearly_baseload_correlations_dict[year] = yearly_corr

    year_raw = raw_multi_timeseries_df[raw_multi_timeseries_df.index.year == year]
    yearly_corr = year_raw.corr()
    yearly_corr.index.names = [f"raw_{year}"]
    yearly_corr.reset_index(inplace=True)
    yearly_raw_correlations_dict[year] = yearly_corr

    yearly_ratio = ratio_multi_timeseries_df[ratio_multi_timeseries_df.index.year == year]
    yearly_corr = yearly_ratio.corr()
    yearly_corr.index.names = [f"raw_{year}"]
    yearly_corr.reset_index(inplace=True)
    yearly_ratio_correlations_dict[year] = yearly_corr

    yearly_return = return_multi_timeseries_df[return_multi_timeseries_df.index.year == year]
    yearly_corr = yearly_return.corr()
    yearly_corr.index.names = [f"raw_{year}"]
    yearly_corr.reset_index(inplace=True)
    yearly_return_correlations_dict[year] = yearly_corr

In [55]:
# MONTH DATA CORRELATIONS
monthly_detrended_correlations_dict = {}
monthly_baseload_correlations_dict = {}
monthly_raw_correlations_dict = {}
monthly_ratio_correlations_dict = {}
monthly_return_correlations_dict = {}

for year in years_in_data:
    year_detrended = detrended_multi_timeseries_df[detrended_multi_timeseries_df.index.year == year]
    year_baseload = baseload_multi_timeseries_df[baseload_multi_timeseries_df.index.year == year]
    year_raw = raw_multi_timeseries_df[raw_multi_timeseries_df.index.year == year]
    yearly_ratio = ratio_multi_timeseries_df[ratio_multi_timeseries_df.index.year == year]
    yearly_return = return_multi_timeseries_df[return_multi_timeseries_df.index.year == year]

    months_in_data = year_detrended.index.month.unique()
    for month in months_in_data:
        month_detrended = year_detrended[year_detrended.index.month == month]
        month_baseload = year_baseload[year_baseload.index.month == month]
        month_raw = year_raw[year_raw.index.month == month]
        month_ratio = yearly_ratio[yearly_ratio.index.month == month]
        month_return = yearly_return[yearly_return.index.month == month]

        monthly_corr = month_detrended.corr()
        monthly_corr.reset_index(inplace=True)
        monthly_detrended_correlations_dict[(year, month)] = monthly_corr

        monthly_corr = month_baseload.corr()
        monthly_corr.reset_index(inplace=True)
        monthly_baseload_correlations_dict[(year, month)] = monthly_corr

        monthly_corr = month_raw.corr()
        monthly_corr.reset_index(inplace=True)
        monthly_raw_correlations_dict[(year, month)] = monthly_corr

        monthly_corr = month_ratio.corr()
        monthly_corr.reset_index(inplace=True)
        monthly_ratio_correlations_dict[(year, month)] = monthly_corr
        
        monthly_corr = month_return.corr()
        monthly_corr.reset_index(inplace=True)
        monthly_return_correlations_dict[(year, month)] = monthly_corr

In [56]:
#CORRELATIONS OF SPECIFIC HOURS

hourly_detrended_correlations_dict = {}
hourly_baseload_correlations_dict = {}
hourly_raw_correlations_dict = {}
hourly_ratio_correlations_dict = {}
hourly_return_correlations_dict = {}

for hour in multi_timeseries_df.Hour.unique():
    hour_detrended = detrended_multi_timeseries_df[detrended_multi_timeseries_df.index.hour == hour]
    hour_baseload = baseload_multi_timeseries_df[baseload_multi_timeseries_df.index.hour == hour]
    hour_ratio = ratio_multi_timeseries_df[ratio_multi_timeseries_df.index.hour == hour]
    hour_raw = raw_multi_timeseries_df[raw_multi_timeseries_df.index.hour == hour]
    hour_return = return_multi_timeseries_df[return_multi_timeseries_df.index.hour == hour]

    hourly_corr = hour_detrended.corr()
    hourly_corr.reset_index(inplace=True)
    hourly_detrended_correlations_dict[hour] = hourly_corr

    hourly_corr = hour_baseload.corr()
    hourly_corr.reset_index(inplace=True)
    hourly_baseload_correlations_dict[hour] = hourly_corr

    hourly_corr = hour_ratio.corr()
    hourly_corr.reset_index(inplace=True)
    hourly_ratio_correlations_dict[hour] = hourly_corr

    hourly_corr = hour_raw.corr()
    hourly_corr.reset_index(inplace=True)
    hourly_raw_correlations_dict[hour] = hourly_corr
    
    hourly_corr = hour_raw.corr()
    hourly_corr.reset_index(inplace=True)
    hourly_return_correlations_dict[hour] = hourly_corr

In [57]:

# HOURLY BY MONTH
hourly_by_month_detrended_correlations_dict = {}
hourly_by_month_baseload_correlations_dict = {}
hourly_by_month_raw_correlations_dict = {}
hourly_by_month_ratio_correlations_dict = {}
hourly_by_month_return_correlations_dict = {}

for month in range(1, 13):
    month_detrended = detrended_multi_timeseries_df[detrended_multi_timeseries_df.index.month == month]
    month_baseload = baseload_multi_timeseries_df[baseload_multi_timeseries_df.index.month == month]
    month_raw = raw_multi_timeseries_df[raw_multi_timeseries_df.index.month == month]
    month_ratio = ratio_multi_timeseries_df[ratio_multi_timeseries_df.index.month == month]
    month_return = return_multi_timeseries_df[return_multi_timeseries_df.index.month == month]
    month_return = return_multi_timeseries_df[return_multi_timeseries_df.index.month == month]

    for hour in multi_timeseries_df.Hour.unique():
        hour_detrended = month_detrended[month_detrended.index.hour == hour]
        hour_baseload = month_baseload[month_baseload.index.hour == hour]
        hour_ratio = month_ratio[month_ratio.index.hour == hour]
        hour_raw = month_raw[month_raw.index.hour == hour]
        hour_return = month_return[month_return.index.hour == hour]

        hourly_corr = hour_detrended.corr()
        hourly_corr.reset_index(inplace=True)
        hourly_by_month_detrended_correlations_dict[(month, hour)] = hourly_corr

        hourly_corr = hour_baseload.corr()
        hourly_corr.reset_index(inplace=True)
        hourly_by_month_baseload_correlations_dict[(month, hour)] = hourly_corr

        hourly_corr = hour_ratio.corr()
        hourly_corr.reset_index(inplace=True)
        hourly_by_month_ratio_correlations_dict[(month, hour)] = hourly_corr

        hourly_corr = hour_raw.corr()
        hourly_corr.reset_index(inplace=True)
        hourly_by_month_raw_correlations_dict[(month, hour)] = hourly_corr
        
        
        hourly_corr = hour_return.corr()
        hourly_corr.reset_index(inplace=True)
        hourly_by_month_return_correlations_dict[(month, hour)] = hourly_corr
        

In [58]:

def yearly_correlations(yearly_dataframe_dict):
    yearly_corr = pd.DataFrame()

    for column in range(1, 7):
        i = 1
        values = []
        column_names = []

        for year in years_in_data:

            for row in range(column, 7):
                name_1 = yearly_dataframe_dict[year].iloc[row, 0]
                name_2 = yearly_dataframe_dict[year].columns[column]
                full_name = [f"{name_1}_{name_2}"]

                value = yearly_dataframe_dict[year].iloc[row, column]
                values = values + [value]
                column_names += full_name

        values = np.array(values)

        values = np.array_split(values, 3) #4

        corr_part = pd.DataFrame(data=values, columns=column_names[: (7 - column)])
        yearly_corr = pd.concat([yearly_corr, corr_part], axis=1)

    return yearly_corr


def monthly_correlations(yearly_dataframe_dict):
    monthly_corr = pd.DataFrame()

    for column in range(1, 7):
        i = 1
        values = []
        column_names = []

        for year_month in yearly_dataframe_dict.keys():
            for row in range(column, 7):
                name_1 = yearly_dataframe_dict[year_month].iloc[row, 0]
                name_2 = yearly_dataframe_dict[year_month].columns[column]
                full_name = [f"{name_1}_{name_2}"]

                value = yearly_dataframe_dict[year_month].iloc[row, column]
                values = values + [value]
                column_names += full_name

        values = np.array(values)
        values = np.array_split(values, 25) #14
     

        corr_part = pd.DataFrame(data=values, columns=column_names[: (7 - column)])
        monthly_corr = pd.concat([monthly_corr, corr_part], axis=1)

    return monthly_corr


def hourly_correlations(hourly_dataframe_dict):
    monthly_corr = pd.DataFrame()

    for column in range(1, 7):
        i = 1
        values = []
        column_names = []

        for hour in hourly_dataframe_dict.keys():
            for row in range(column, 7):
                name_1 = hourly_dataframe_dict[hour].iloc[row, 0]
                name_2 = hourly_dataframe_dict[hour].columns[column]
                full_name = [f"{name_1}_{name_2}"]

                value = hourly_dataframe_dict[hour].iloc[row, column]
                values = values + [value]
                column_names += full_name

        values = np.array(values)
        values = np.array_split(values, 24)

        corr_part = pd.DataFrame(data=values, columns=column_names[: (7 - column)])
        monthly_corr = pd.concat([monthly_corr, corr_part], axis=1)

    return monthly_corr


def hourly_by_month_correlations(hourly_by_months_dict):
    yearly_corr = pd.DataFrame()

    for column in range(1, 7):
        i = 1
        values = []
        column_names = []

        for month_hour in hourly_by_months_dict.keys():
            for row in range(column, 7):
                name_1 = hourly_by_months_dict[month_hour].iloc[row, 0]
                name_2 = hourly_by_months_dict[month_hour].columns[column]
                full_name = [f"{name_1}_{name_2}"]

                value = hourly_by_months_dict[month_hour].iloc[row, column]
                values = values + [value]
                column_names += full_name

        values = np.array(values)
        values = np.array_split(values, 288)

        corr_part = pd.DataFrame(data=values, columns=column_names[: (7 - column)])
        yearly_corr = pd.concat([yearly_corr, corr_part], axis=1)

    return yearly_corr

In [59]:
years_in_data

Int64Index([2020, 2021, 2022], dtype='int64', name='datetime')

In [60]:
# MAKING IT LOOK NICE
raw_correlations_yearly = yearly_correlations(yearly_raw_correlations_dict)

detrended_correlations_yearly = yearly_correlations(yearly_detrended_correlations_dict)
detrended_correlations_yearly.columns = [
    name.replace("_detrended", "", 1) for name in detrended_correlations_yearly.columns
]

baseload_correlations_yearly = yearly_correlations(yearly_baseload_correlations_dict)
baseload_correlations_yearly.columns = [
    name.replace("_baseload", "", 1) for name in baseload_correlations_yearly.columns
]

ratio_correlations_yearly = yearly_correlations(yearly_ratio_correlations_dict)
ratio_correlations_yearly.columns = [
        name.replace("_ratio", "", 1) for name in ratio_correlations_yearly.columns
    ]
    
return_correlations_yearly = yearly_correlations(yearly_return_correlations_dict)
return_correlations_yearly.columns = [
        name.replace("_return", "", 1) for name in return_correlations_yearly.columns
    ]

In [61]:
# ------ Monthly nice
raw_correlations_monthly = monthly_correlations(monthly_raw_correlations_dict)

detrended_correlations_monthly = monthly_correlations(monthly_detrended_correlations_dict)
detrended_correlations_monthly.columns = [
    name.replace("_detrended", "", 1) for name in detrended_correlations_monthly.columns
]

baseload_correlations_monthly = monthly_correlations(monthly_baseload_correlations_dict)
baseload_correlations_monthly.columns = [
    name.replace("_baseload", "", 1) for name in baseload_correlations_monthly.columns
]

ratio_correlations_monhtly = monthly_correlations(monthly_ratio_correlations_dict)
ratio_correlations_monhtly.columns = [
    name.replace("_ratio", "", 1) for name in ratio_correlations_monhtly.columns
]

return_correlations_monthly = monthly_correlations(monthly_return_correlations_dict)
return_correlations_monthly.columns = [
        name.replace("_return", "", 1) for name in return_correlations_monthly.columns
    ]
    
# ------ Hourly DF
raw_correlations_hourly = hourly_correlations(hourly_raw_correlations_dict)

detrended_correlations_hourly = hourly_correlations(hourly_detrended_correlations_dict)
detrended_correlations_hourly.columns = [
    name.replace("_detrended", "", 1) for name in detrended_correlations_hourly.columns
]

baseload_correlations_hourly = hourly_correlations(hourly_baseload_correlations_dict)
baseload_correlations_hourly.columns = [
    name.replace("_baseload", "", 1) for name in baseload_correlations_hourly.columns
]

ratio_correlations_hourly = hourly_correlations(hourly_ratio_correlations_dict)
ratio_correlations_hourly.columns = [
    name.replace("_ratio", "", 1) for name in ratio_correlations_hourly.columns
]

return_correlations_hourly = hourly_correlations(hourly_return_correlations_dict)
return_correlations_hourly.columns = [
    name.replace("_return", "", 1) for name in return_correlations_hourly.columns
]

In [62]:
raw_correlations_yearly

,BE_NL,AT_NL,DE_NL,FR_NL,CZ_NL,UK_NL,AT_BE,DE_BE,FR_BE,CZ_BE,...,DE_AT,FR_AT,CZ_AT,UK_AT,FR_DE,CZ_DE,UK_DE,CZ_FR,UK_FR,UK_CZ
0,0.838415,0.003366,0.024490,0.015362,0.467668,-0.004237,0.385505,0.565401,0.392891,0.873856,...,0.702001,0.494841,0.622189,0.285640,0.697394,0.895092,0.412912,0.623466,0.605308,0.363194
1,0.831368,-0.011224,0.014943,0.012360,0.453360,0.005930,0.387084,0.568083,0.404203,0.872239,...,0.713087,0.503233,0.630726,0.313275,0.708962,0.898002,0.435600,0.637427,0.583577,0.390915
2,0.832846,0.009223,0.003355,0.010407,0.449924,0.009805,0.406514,0.556296,0.404390,0.869034,...,0.720587,0.521974,0.647658,0.309535,0.714971,0.894571,0.412754,0.643171,0.588680,0.373001


In [63]:

# ------
raw_correlations_hourly_by_month = hourly_by_month_correlations(hourly_by_month_raw_correlations_dict)

detrended_correlations_hourly_by_month = hourly_by_month_correlations(
    hourly_by_month_detrended_correlations_dict
)
detrended_correlations_hourly_by_month.columns = [
    name.replace("_detrended", "", 1) for name in detrended_correlations_hourly_by_month.columns
]

baseload_correlations_hourly_by_month = hourly_by_month_correlations(
    hourly_by_month_baseload_correlations_dict)
baseload_correlations_hourly_by_month.columns = [
    name.replace("_baseload", "", 1) for name in baseload_correlations_hourly_by_month.columns
]

ratio_correlations_hourly_by_month = hourly_by_month_correlations(hourly_by_month_ratio_correlations_dict)
ratio_correlations_hourly_by_month.columns = [
    name.replace("_ratio", "", 1) for name in ratio_correlations_hourly_by_month.columns
]

return_correlations_hourly_by_month = hourly_by_month_correlations(hourly_by_month_return_correlations_dict)
return_correlations_hourly_by_month.columns = [
    name.replace("_return", "", 1) for name in return_correlations_hourly_by_month.columns
]

In [64]:
raw_correlations_hourly_by_month.head(3)

,BE_NL,AT_NL,DE_NL,FR_NL,CZ_NL,UK_NL,AT_BE,DE_BE,FR_BE,CZ_BE,...,DE_AT,FR_AT,CZ_AT,UK_AT,FR_DE,CZ_DE,UK_DE,CZ_FR,UK_FR,UK_CZ
0,0.825358,0.336232,-0.011327,0.084978,0.432527,0.013841,0.661634,0.555225,0.441435,0.866054,...,0.676480,0.503568,0.758831,0.136638,0.656614,0.896664,0.205050,0.629678,0.423161,0.191017
1,0.879325,-0.076340,-0.153049,-0.058602,0.455907,-0.053912,0.274012,0.336031,0.183983,0.824740,...,0.719591,0.289653,0.602764,0.240827,0.497689,0.809766,0.331342,0.413443,0.623887,0.266410
2,0.734279,0.011907,0.020801,0.048132,0.350649,-0.280531,0.577502,0.693974,0.586597,0.893220,...,0.837897,0.784342,0.788809,0.502564,0.812870,0.943598,0.511143,0.777363,0.707835,0.385891


In [65]:
month= []
hour = []
for i in range(1,13):
    month=month+[i]*24
    hour= hour+list(range(1, 25))
    
raw_correlations_hourly_by_month['Month']=month
raw_correlations_hourly_by_month['Hour']=hour

return_correlations_hourly_by_month['Month']=month
return_correlations_hourly_by_month['Hour']=hour


raw_correlations_hourly_by_month=raw_correlations_hourly_by_month[['Month', 'Hour','BE_NL', 'AT_NL', 'DE_NL', 'FR_NL', 'CZ_NL', 'UK_NL', 'AT_BE', 'DE_BE',
       'FR_BE', 'CZ_BE', 'UK_BE', 'DE_AT', 'FR_AT', 'CZ_AT', 'UK_AT', 'FR_DE','CZ_DE', 'UK_DE', 'CZ_FR', 'UK_FR', 'UK_CZ']]


return_correlations_hourly_by_month=return_correlations_hourly_by_month[['Month', 'Hour','return_BE_NL', 'return_AT_NL', 'return_DE_NL', 'return_FR_NL',
       'return_CZ_NL', 'return_UK_NL', 'return_AT_BE', 'return_DE_BE','return_FR_BE', 'return_CZ_BE', 'return_UK_BE', 'return_DE_AT',
       'return_FR_AT', 'return_CZ_AT', 'return_UK_AT', 'return_FR_DE','return_CZ_DE', 'return_UK_DE', 'return_CZ_FR', 'return_UK_FR',
       'return_UK_CZ']]

In [66]:
raw_correlations_hourly_by_month.to_csv('./raw_correlations_hourly_by_month.csv')
ratio_correlations_hourly_by_month.to_csv('./ratio_correlations_hourly_by_month.csv')
return_correlations_hourly_by_month.to_csv('./return_correlations_hourly_by_month.csv')
multi_timeseries_df.to_csv('./multi_timeseries_df.csv')